In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path
from datetime import datetime

notebooks_dir = Path.cwd() if Path.cwd().name == 'notebooks' else Path.cwd() / 'notebooks'
sql_file = notebooks_dir / 'database.sql'
output_dir = notebooks_dir / 'output'
output_dir.mkdir(parents=True, exist_ok=True)

# Load database
temp_db = output_dir / 'temp_database.db'
if temp_db.exists():
    temp_db.unlink()

conn = sqlite3.connect(str(temp_db))
with open(sql_file, 'r', encoding='utf-8') as f:
    conn.executescript(f.read())
conn.commit()
print("DB loaded")

In [ ]:
# Get all events
events = pd.read_sql_query("SELECT id, name FROM events ORDER BY name", conn)
print(f"Found {len(events)} events")

In [ ]:
# Registrations without participants
registrations = pd.read_sql_query("""
    SELECT 
        e.name as event_name,
        s.name as school_name,
        s.location as school_location,
        r.team_name,
        f.faculty_name as registered_by,
        f.mobile_number
    FROM registrations r
    JOIN events e ON r.event_id = e.id
    JOIN schools s ON r.school_id = s.id
    JOIN faculty f ON r.registered_by_faculty_id = f.id
    LEFT JOIN registration_participants rp ON r.id = rp.registration_id
    WHERE rp.id IS NULL
    ORDER BY e.name, s.name
""", conn)

print(f"Registrations without participants: {len(registrations)}")
display(registrations)

In [ ]:
# Students and faculty counts per school
school_summary = pd.read_sql_query("""
    SELECT 
        s.name as school_name,
        s.location as school_location,
        (SELECT COUNT(*) FROM students WHERE school_id = s.id) as total_students,
        (SELECT COUNT(*) FROM students st 
         WHERE st.school_id = s.id 
         AND NOT EXISTS (
             SELECT 1 FROM registration_participants rp 
             WHERE rp.participant_id = st.id AND rp.participant_type = 'student'
         )) as unassigned_students,
        (SELECT COUNT(*) FROM faculty WHERE school_id = s.id) as total_faculty,
        (SELECT COUNT(*) FROM faculty f 
         WHERE f.school_id = s.id 
         AND NOT EXISTS (
             SELECT 1 FROM registration_participants rp 
             WHERE rp.participant_id = f.id AND rp.participant_type = 'faculty'
         )) as unassigned_faculty
    FROM schools s
    WHERE EXISTS (SELECT 1 FROM students WHERE school_id = s.id)
       OR EXISTS (SELECT 1 FROM faculty WHERE school_id = s.id)
    ORDER BY s.name
""", conn)

school_summary['total_students'] = school_summary['total_students'].astype(int)
school_summary['unassigned_students'] = school_summary['unassigned_students'].astype(int)
school_summary['total_faculty'] = school_summary['total_faculty'].astype(int)
school_summary['unassigned_faculty'] = school_summary['unassigned_faculty'].astype(int)

print(f"Schools: {len(school_summary)}")
display(school_summary)

In [ ]:
# Query functions
def get_individual_event_data(event_id, conn):
    query = """
    SELECT 
        e.name as event_name,
        e.age_category,
        r.id as registration_id,
        r.team_name,
        s.school_code,
        s.name as school_name,
        s.location as school_location,
        f.faculty_name as registered_by,
        f.mobile_number,
        GROUP_CONCAT(
            CASE WHEN rp.participant_type = 'student' THEN st.student_name ELSE fa.faculty_name END, ', '
        ) as participants_name
    FROM registrations r
    JOIN events e ON r.event_id = e.id
    JOIN schools s ON r.school_id = s.id
    JOIN faculty f ON r.registered_by_faculty_id = f.id
    LEFT JOIN registration_participants rp ON r.id = rp.registration_id
    LEFT JOIN students st ON rp.participant_id = st.id AND rp.participant_type = 'student'
    LEFT JOIN faculty fa ON rp.participant_id = fa.id AND rp.participant_type = 'faculty'
    WHERE r.event_id = ?
    GROUP BY r.id, e.name, e.age_category, r.team_name, s.school_code, s.name, s.location, f.faculty_name, f.mobile_number
    ORDER BY s.name, r.id
    """
    return pd.read_sql_query(query, conn, params=(event_id,))

def get_group_event_data(event_id, conn):
    query = """
    SELECT 
        e.name as event_name,
        e.age_category,
        r.id as registration_id,
        r.team_name,
        s.school_code,
        s.name as school_name,
        s.location as school_location,
        f.faculty_name as registered_by,
        f.mobile_number,
        GROUP_CONCAT(
            CASE WHEN rp.participant_type = 'student' THEN st.student_name ELSE fa.faculty_name END, ', '
        ) as participants_name
    FROM registrations r
    JOIN events e ON r.event_id = e.id
    JOIN schools s ON r.school_id = s.id
    JOIN faculty f ON r.registered_by_faculty_id = f.id
    LEFT JOIN registration_participants rp ON r.id = rp.registration_id
    LEFT JOIN students st ON rp.participant_id = st.id AND rp.participant_type = 'student'
    LEFT JOIN faculty fa ON rp.participant_id = fa.id AND rp.participant_type = 'faculty'
    WHERE r.event_id = ?
    GROUP BY r.id, e.name, e.age_category, r.team_name, s.school_code, s.name, s.location, f.faculty_name, f.mobile_number
    ORDER BY s.name, r.id
    """
    return pd.read_sql_query(query, conn, params=(event_id,))

In [ ]:
# Generate Excel files
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter

events_with_type = pd.read_sql_query("SELECT id, name, event_type FROM events ORDER BY event_type, name", conn)
individual_events = events_with_type[events_with_type['event_type'] == 'Individual']
group_events = events_with_type[events_with_type['event_type'].isin(['Group', 'Combined', 'Special'])]

def clean_sheet_name(name):
    return name.replace('/', '_').replace('\\', '_').replace('?', '_').replace('*', '_').replace('[', '_').replace(']', '_').replace(':', '_').replace('(', '_').replace(')', '_')[:31]

def format_worksheet(worksheet, df):
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF", size=11)
    
    for cell in worksheet[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
    
    for idx, column in enumerate(df.columns, 1):
        column_letter = get_column_letter(idx)
        max_length = max(len(str(column)), df[column].astype(str).map(len).max() if len(df) > 0 else 0)
        worksheet.column_dimensions[column_letter].width = min(max(max_length + 2, 10), 50)
    
    worksheet.freeze_panes = "A2"
    worksheet.row_dimensions[1].height = 25

# Individual Events file
individual_file = output_dir / f'individual_events_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
with pd.ExcelWriter(individual_file, engine='openpyxl') as writer:
    for _, event in individual_events.iterrows():
        df = get_individual_event_data(event['id'], conn)
        sheet_name = clean_sheet_name(event['name'])
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        format_worksheet(writer.sheets[sheet_name], df)
        print(f"{event['name']}: {len(df)} rows")

print(f"\nSaved: {individual_file}")

# Group Events file
group_file = output_dir / f'group_events_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
with pd.ExcelWriter(group_file, engine='openpyxl') as writer:
    for _, event in group_events.iterrows():
        df = get_group_event_data(event['id'], conn)
        sheet_name = clean_sheet_name(event['name'])
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        format_worksheet(writer.sheets[sheet_name], df)
        print(f"{event['name']}: {len(df)} rows")

print(f"\nSaved: {group_file}")
conn.close()